In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypertension"
cohort = "GSE128381"

# Input paths
in_trait_dir = "../../input/GEO/Hypertension"
in_cohort_dir = "../../input/GEO/Hypertension/GSE128381"

# Output paths
out_data_file = "../../output/preprocess/Hypertension/GSE128381.csv"
out_gene_data_file = "../../output/preprocess/Hypertension/gene_data/GSE128381.csv"
out_clinical_data_file = "../../output/preprocess/Hypertension/clinical_data/GSE128381.csv"
json_path = "../../output/preprocess/Hypertension/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide a corrected implementation that properly handles the sample characteristics dictionary:

```python
# 1. Assess gene expression data availability
# Based on the series title and summary, this dataset appears to contain gene expression data from RNA microarray
is_gene_available = True

# 2. Variable availability and data type conversion
# 2.1. Identify rows for trait, age, and gender in the sample characteristics dictionary
# For hypertension, row 14 contains gestational hypertension information
trait_row = 14

# Age information is available in row 10 (maternal age)
age_row = 10

# Gender information is available in row 6
gender_row = 6

# 2.2. Define conversion functions for each variable

def convert_trait(value):
    """Convert hypertension status to binary."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0: no, 1: yes)
    if value == '0 (no)':
        return 0
    elif value == '1 (yes)':
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract the numeric value
    try:
        # Extract just the number
        if '(year)' in value:
            value = value.replace('(year)', '').strip()
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0: female, 1: male)."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save metadata about usability
# Check if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    sample_chars = {0: ['tissue: Placenta'], 
                   1: ['labeling date: 6/12/2017', 'labeling date: 4/21/2017', 'labeling date: 6/9/2017', 'labeling date: 5/29/2017', 'labeling date: 6/7/2017', 'labeling date: 6/13/2017', 'labeling date: 6/15/2017', 'labeling date: 6/14/2017', 'labeling date: 2/20/2017', 'labeling date: 8/15/2017'], 
                   2: ['hybridization date: 6/28/2017', 'hybridization date: 4/24/2017', 'hybridization date: 6/27/2017', 'hybridization date: 6/21/2017', 'hybridization date: 6/26/2017', 'hybridization date: 7/3/2017', 'hybridization date: 7/12/2017', 'hybridization date: 7/4/2017', 'hybridization date: 2/22/2017', 'hybridization date: 7/10/2017', 'hybridization date: 8/17/2017'], 
                   3: ['date delivery: 1/24/2014', 'date delivery: 1/25/2014', 'date delivery: 2/15/2014', 'date delivery: 2/7/2014', 'date delivery: 4/24/2014', 'date delivery: 3/9/2014', 'date delivery: 3/14/2014', 'date delivery: 4/13/2014', 'date delivery: 5/2/2014', 'date delivery: 5/22/2014', 'date delivery: 5/28/2014', 'date delivery: 7/14/2014', 'date delivery: 7/17/2014', 'date delivery: 8/14/2014', 'date delivery: 9/5/2014', 'date delivery: 9/12/2014', 'date delivery: 9/15/2014', 'date delivery: 9/24/2014', 'date delivery: 10/3/2014', 'date delivery: 10/31/2014', 'date delivery: 10/10/2014', 'date delivery: 10/24/2014', 'date delivery: 11/6/2014', 'date delivery: 11/7/2014', 'date delivery: 12/5/2014', 'date delivery: 2/13/2015', 'date delivery: 2/24/2015', 'date delivery: 5/1/2015', 'date delivery: 2/28/2015', 'date delivery: 3/6/2015'], 
                   4: ['maternal pre-pregnancy bmi: 23', 'maternal pre-pregnancy bmi: 31.2', 'maternal pre-pregnancy bmi: 18.4', 'maternal pre-pregnancy bmi: 25.3', 'maternal pre-pregnancy bmi: 22.4', 'maternal pre-pregnancy bmi: 19.7', 'maternal pre-pregnancy bmi: 22', 'maternal pre-pregnancy bmi: 21.1', 'maternal pre-pregnancy bmi: 18.7', 'maternal pre-pregnancy bmi: 34.3', 'maternal pre-pregnancy bmi: 39.3', 'maternal pre-pregnancy bmi: 19.3', 'maternal pre-pregnancy bmi: 24.3', 'maternal pre-pregnancy bmi: 28.4', 'maternal pre-pregnancy bmi: 47.2', 'maternal pre-pregnancy bmi: 18.2', 'maternal pre-pregnancy bmi: 23.4', 'maternal pre-pregnancy bmi: 27.5', 'maternal pre-pregnancy bmi: 19.6', 'maternal pre-pregnancy bmi: 26.2', 'maternal pre-pregnancy bmi: 19.5', 'maternal pre-pregnancy bmi: 27.1', 'maternal pre-pregnancy bmi: 25.4', 'maternal pre-pregnancy bmi: 29', 'maternal pre-pregnancy bmi: 26.6', 'maternal pre-pregnancy bmi: 16.8', 'maternal pre-pregnancy bmi: 24.9', 'maternal pre-pregnancy bmi: 24.7', 'maternal pre-pregnancy bmi: 29.9', 'maternal pre-pregnancy bmi: 22.2'], 
                   5: ['birth weight (gram): 3575', 'birth weight (gram): 3635', 'birth weight (gram): 2415', 'birth weight (gram): 3725', 'birth weight (gram): 3965', 'birth weight (gram): 3735', 'birth weight (gram): 2775', 'birth weight (gram): 1930', 'birth weight (gram): 2890', 'birth weight (gram): 3240', 'birth weight (gram): 2700', 'birth weight (gram): 2590', 'birth weight (gram): 3135', 'birth weight (gram): 1765', 'birth weight (gram): 2975', 'birth weight (gram): 3100', 'birth weight (gram): 3055',


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Analyzing sample data
import os
import json
import pandas as pd
from typing import Optional, Dict, Any, Callable

# Check for various possible locations/naming of the clinical data
possible_clinical_files = [
    os.path.join(in_cohort_dir, "sample_characteristics.csv"),
    os.path.join(in_cohort_dir, "characteristics.csv"),
    os.path.join(in_cohort_dir, "clinical_data.csv"),
    os.path.join(in_cohort_dir, "samples.csv"),
    os.path.join(in_cohort_dir, "metadata.csv")
]

clinical_data = pd.DataFrame()
found_clinical_file = False

for file_path in possible_clinical_files:
    if os.path.exists(file_path):
        clinical_data = pd.read_csv(file_path, index_col=0)
        print(f"Sample characteristics data loaded successfully from {file_path}")
        print(f"Shape of clinical_data: {clinical_data.shape}")
        
        # Check the first few rows to see column names and data structure
        print("\nPreview of clinical_data:")
        print(clinical_data.head())
        
        # Check unique values for each row to determine data availability
        print("\nUnique values for each row:")
        for i, row in clinical_data.iterrows():
            print(f"Row {i}: {row.unique()}")
        
        found_clinical_file = True
        break

if not found_clinical_file:
    print("Clinical data file not found in any expected location. Checking for any CSV files in the directory.")
    
    # Look for any CSV files in the directory
    all_files = os.listdir(in_cohort_dir)
    csv_files = [f for f in all_files if f.endswith('.csv')]
    
    if csv_files:
        print(f"Found the following CSV files: {csv_files}")
        # Try to load the first CSV file found
        first_csv = os.path.join(in_cohort_dir, csv_files[0])
        try:
            clinical_data = pd.read_csv(first_csv, index_col=0)
            print(f"Loaded {first_csv} as clinical data")
            print(f"Shape of clinical_data: {clinical_data.shape}")
            print("\nPreview of clinical_data:")
            print(clinical_data.head())
            found_clinical_file = True
        except Exception as e:
            print(f"Error loading {first_csv}: {e}")
    else:
        print(f"No CSV files found in {in_cohort_dir}")

# Function to help extract values after colon
def extract_value_after_colon(text):
    if pd.isna(text):
        return None
    if ':' in str(text):
        return str(text).split(':', 1)[1].strip()
    return str(text).strip()

# 1. Gene Expression Data Availability
# Assuming gene expression data is likely available unless we find evidence to the contrary
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Identifying rows for trait, age, and gender data
trait_row = None
age_row = None
gender_row = None

# Check if clinical data is available before proceeding
if not clinical_data.empty:
    # Checking sample characteristics for trait data
    for i, row in clinical_data.iterrows():
        unique_values = row.unique()
        row_str = ' '.join([str(v) for v in unique_values if pd.notna(v)])
        
        # Check for hypertension-related terms
        if 'hypertension' in row_str.lower() or 'blood pressure' in row_str.lower() or 'bp' in row_str.lower():
            trait_row = i
            print(f"Found trait data in row {i}: {unique_values}")
        
        # Check for age-related terms
        if 'age' in row_str.lower() or 'year' in row_str.lower():
            age_row = i
            print(f"Found age data in row {i}: {unique_values}")
        
        # Check for gender-related terms
        if 'gender' in row_str.lower() or 'sex' in row_str.lower() or 'male' in row_str.lower() or 'female' in row_str.lower():
            gender_row = i
            print(f"Found gender data in row {i}: {unique_values}")
else:
    print("No clinical data available. Cannot identify trait, age, or gender information.")

# Define conversion functions
def convert_trait(value):
    if pd.isna(value):
        return None
    
    value = extract_value_after_colon(value).lower()
    
    if 'yes' in value or 'hypertension' in value or 'high' in value:
        return 1
    elif 'no' in value or 'normal' in value or 'control' in value:
        return 0
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    
    value = extract_value_after_colon(value)
    
    # Try to extract number from the string
    import re
    numbers = re.findall(r'\d+\.?\d*', value)
    if numbers:
        return float(numbers[0])
    return None

def convert_gender(value):
    if pd.isna(value):
        return None
    
    value = extract_value_after_colon(value).lower()
    
    if 'male' in value or 'm' == value:
        return 1
    elif 'female' in value or 'f' == value:
        return 0
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the clinical dataframe
    print("\nPreview of processed clinical data:")
    preview = preview_df(clinical_df)
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("No trait data found. Skipping clinical feature extraction.")
    
    # Even if we don't have trait data, we should still create the output directory
    # in case we need to save other data files later
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)


### Step 4: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 5: Gene Identifier Review

In [ ]:
# Reviewing the gene identifiers
# The identifiers starting with "A_19_P" appear to be Agilent microarray probe IDs,
# not standard human gene symbols like BRCA1, TP53, etc.
# These need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine mapping columns
# From the gene annotation preview, we can see:
# - The 'ID' column contains probe IDs like 'A_23_P117082' matching the expression data index
# - The 'GENE_SYMBOL' column contains human gene symbols like 'HEBP1', 'KCNE4', etc.

# 2. Get gene mapping dataframe by extracting relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply gene mapping to convert probe-level data to gene-level data
# This handles many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Display first few genes to verify mapping worked correctly
print("First 10 genes after mapping:")
print(gene_data.index[:10])
print("\nShape of gene expression data after mapping:")
print(gene_data.shape)


### Step 8: Data Normalization and Linking

In [ ]:
# 1. First, we need to re-obtain the clinical data from the matrix file since it wasn't successfully loaded in previous steps
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Re-extract background information and clinical data from the matrix file
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Get unique values for each row to identify trait, age, and gender rows
sample_chars = get_unique_values_by_row(clinical_data)

# Identify rows for hypertension trait, age, and gender based on the sample characteristics
trait_row = 14  # 'gestational hypertension: 0 (no)' / 'gestational hypertension: 1 (yes)'
age_row = 10    # 'maternal age (year): XX'
gender_row = 6  # 'Sex: Male' / 'Sex: Female'

# Define conversion functions for each variable
def convert_trait(value):
    """Convert hypertension status to binary."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0: no, 1: yes)
    if '0 (no)' in value:
        return 0
    elif '1 (yes)' in value:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract the numeric value
    try:
        # Parse the year value
        year_value = value.replace('(year)', '').strip()
        return float(year_value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0: female, 1: male)."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# Extract clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the extracted clinical features to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Re-extract the gene expression data from the matrix file
gene_data_raw = get_genetic_data(matrix_file)

# Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# Get gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data_raw, gene_mapping)

# Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data before missing value handling: {linked_data.shape}")

# Handle missing values systematically
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after missing value handling: {linked_data.shape}")

# Check if the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# Validate and save cohort information
note = "Dataset containing maternal gestational hypertension information and placental gene expression data."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to quality issues.")